# HEPData script for SUS-21-008 to make .yaml files

Input root files from: https://pmeiring.web.cern.ch/pmeiring/EWK_SUSY_combination/20230301_SummariesApproval/

In [61]:
from hepdata_lib import Submission, Table, RootFileReader, Variable, Uncertainty
import ROOT, pickle
submission = Submission()
submission.read_abstract("abstract.txt")
submission.add_link("Webpage with all figures and tables", "https://cms-results.web.cern.ch/cms-results/public-results/publications/SUS-21-008/")
#submission.add_link("arXiv", "Fill later")
#submission.add_record_id(1111, "inspire")

In [62]:
# Extract TGraphErrors and save the variables in a pickle file
def round_(x):
    for i in range(len(x)):
        x[i] = round(x[i], 2)
    return x

def extract_y_values_and_errors(file_path, is_symmetric=False):
    # Open the ROOT file
    tf = ROOT.TFile.Open(file_path)
    if not tf or tf.IsZombie():
        raise RuntimeError(f"Error: Could not open the ROOT file '{file_path}'.")

    # Create a list to hold the variables for each TGraphAsymmErrors
    variable_list = []

    # Loop over all objects in the ROOT file
    key_list = tf.GetListOfKeys()
    for key in key_list:
        obj = key.ReadObj()

        # Check if the object is a TGraphAsymmErrors
        if isinstance(obj, ROOT.TGraphAsymmErrors):
            # Extract the y-values and asymmetric errors
            y_values = [obj.GetY()[i] for i in range(obj.GetN())]
            errors_low = [obj.GetErrorYlow(i) for i in range(obj.GetN())]
            errors_high = [obj.GetErrorYhigh(i) for i in range(obj.GetN())]
            # low errors minus
            errors_low = [i * -1 for i in errors_low]
            round_(y_values)
            print(y_values)
            round_(errors_low)
            round_(errors_high)
         
            var_name = obj.GetName()
            print(var_name)
            variable = Variable(var_name, values=y_values, is_independent=False, is_binned=False)
            #unc = Uncertainty(label=var_name+f"Uncertainty", is_symmetric=False)
            unc = Uncertainty(label="", is_symmetric=False)
            
            uncertain = []
            #Create separate asymmetric uncertainties for each point
            for i, (low, high) in enumerate(zip(errors_low, errors_high)):
                unc.minus = low
                unc.plus = high
                uncertain.append((low,high))
            
            unc.values = uncertain
            variable.add_uncertainty(unc)
            # Add the variable to the list
            variable_list.append(variable)

    # Close the ROOT file
    tf.Close()

    return variable_list


In [63]:
variable_list = extract_y_values_and_errors("hists_fig_6-8.root")
# Save the variables to a file
output_file = "variables.pkl"
with open(output_file, 'wb') as f:
    pickle.dump(variable_list, f)

loaded_variables = None
with open(output_file, 'rb') as f:
    loaded_variables = pickle.load(f)


[92.59, 123.59, 230.06]
SR_2l_ewk_dM20_low_background_errors
[99.0, 139.0, 236.0]
SR_2l_ewk_dM20_low_data_graph
[21.54, 11.97, 18.3, 31.64, 71.33]
SR_2l_ewk_dM20_med_background_errors
[18.0, 11.0, 17.0, 34.0, 71.0]
SR_2l_ewk_dM20_med_data_graph
[9.71, 5.16, 7.55, 11.52, 27.91]
SR_2l_ewk_dM20_high_background_errors
[11.0, 6.0, 5.0, 10.0, 23.0]
SR_2l_ewk_dM20_high_data_graph
[5.19, 3.37, 4.54, 7.16, 15.3]
SR_2l_ewk_dM20_ultra_background_errors
[3.0, 4.0, 5.0, 7.0, 22.0]
SR_2l_ewk_dM20_ultra_data_graph
[4.97, 2.01, 1.26, 1.9, 4.24]
SR_3l_ewk_dM20_low_background_errors
[2.0, 4.0, 1.0, 3.0, 5.0]
SR_3l_ewk_dM20_low_data_graph
[4.95, 2.0, 1.34, 1.73, 2.95]
SR_3l_ewk_dM20_med_background_errors
[4.0, 1.0, 1.0, 3.0, 3.0]
SR_3l_ewk_dM20_med_data_graph
[78.2, 63.55, 91.25, 212.54]
MT2_NLSP125_dM10_low_background_errors
[78.0, 66.0, 110.0, 220.0]
MT2_NLSP125_dM10_low_data_graph
[48.37, 23.88, 16.26, 14.72, 18.11, 32.0]
MT2_NLSP125_dM10_med_background_errors
[48.0, 22.0, 13.0, 22.0, 16.0, 30.0]
MT2_

In [64]:
#The TH1Ds from input files are extracted out of TCanvas locally and hists are read here, adding the variables from TGraphErrors to the table

def getTH1Ds(inputFile, fig, pdf, location, prefix, varss, title, xBin_name, unit):
    table = Table(fig)
    #table.add_image(pdf)
    table.description = title
    table.location = location
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = ["N"]
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    reader = RootFileReader(inputFile)
    #Data = reader.read_hist_1d(prefix+"data")
    
    for var in varss:
        readHist = reader.read_hist_1d(prefix+var)
        var_map = {"prompt_wz":"WZ",
                   "prompt_vv":"VV",
                   "Rares":"Other",
                   "FRfakes_data":"Nonprompt l",
                   "semidd_fakes":"Nonprompt l",
                   "prompt_dy":"DY",
                   "prompt_tt":"tt",
                   "signal_TChiWZ_200_180_norm":"TChiWZ(200,180)",
                   "signal_TSlepSlep_125_115_norm":"TSlepSlep(125,115)"}
        varHist = Variable(var_map[var], is_independent=False, is_binned=False, units="")
        varHist.values = readHist["y"]
        round_(varHist.values)
        table.add_variable(varHist)
    
    
    def find_variable_by_name(variable_list, name):
        for variable in variable_list:
            #print(variable)
            if variable.name == name and "background_errors" in variable.name:
                variable.name = "Total BG"
                return variable
            elif variable.name == name and "data_graph" in variable.name:
                variable.name = "Data"
                return variable
        return None
    
    
    v1 = find_variable_by_name(variable_list, prefix+"background_errors")
    print(v1.name)
    table.add_variable(v1)
    v2 = find_variable_by_name(variable_list, prefix+"data_graph")
    table.add_variable(v2)
    print(v2.name)
    
    X = reader.read_hist_1d(prefix+varss[0])
    xBins = Variable(xBin_name, is_independent=True, is_binned=True, units = unit)
    xBins.values = X["x_edges"]
    table.add_variable(xBins)
    
    
    submission.add_table(table)

In [65]:
#Fig 6

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6a",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_006-a.pdf",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_low_",
    varss = ["prompt_wz","prompt_vv","Rares","FRfakes_data","prompt_dy","prompt_tt","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6b",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_006-b.pdf",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_med_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","prompt_dy","prompt_tt","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6c",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_006-c.pdf",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_high_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","prompt_dy","prompt_tt","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6d",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_006-d.pdf",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_ultra_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","prompt_dy","prompt_tt","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

Total BG
Data
Total BG
Data
Total BG
Data
Total BG
Data


In [66]:
#Fig 7

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 7a",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_007-a.pdf",
    location = "Data on page 10",
    prefix   = "SR_3l_ewk_dM20_low_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 3l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 7b",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_007-b.pdf",
    location = "Data on page 10",
    prefix   = "SR_3l_ewk_dM20_med_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 3l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

Total BG
Data
Total BG
Data


In [67]:
# Fig 8

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8a",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_008-a.pdf",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_low_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","FRfakes_data","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for low p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8b",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_008-b.pdf",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_med_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","semidd_fakes","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for medium p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8c",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_008-c.pdf",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_high_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","semidd_fakes","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for high p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8d",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_008-d.pdf",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_ultra_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","semidd_fakes","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for ultra p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

Total BG
Data
Total BG
Data
Total BG
Data
Total BG
Data


In [80]:
def getTH1Ds_noAdding(inputFile, fig, pdf, location, prefix, varss, title, xBin_name, unit):
    table = Table(fig)
    table.add_image(pdf)
    table.description = title
    table.location = location
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = ["N"]
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    reader = RootFileReader(inputFile)
    
    for var in varss:
        readHist = reader.read_hist_1d(prefix+var)
        var_map = {"prompt_WZ":"WZ",
                   "prompt_VH":"VH",
                   "Rares":"Other",
                   "data_fakes":"Nonprompt l",
                   "semidd_fakes":"Nonprompt l",
                   "prompt_dy":"DY",
                   "prompt_TTX":"ttX",
                   "prompt_ZZ":"ZZ",
                   "convs":"X+photon",
                   "TChiWZ_200_160_norm":"TChiWZ(200,160)",
                   "TChiWZ_600_100_norm":"TChiWZ(600,100)",
                   "TChiWH_150_1_norm":"TChiWH(200,1)",
                   "TChiWH_150_25_norm":"TChiWH(200,25)",
                   "data":"Data",
                   "total":"Total BG"
                  }
        varHist = Variable(var_map[var], is_independent=False, is_binned=False, units="")
        varHist.values = readHist["y"]
        round_(varHist.values)

        if "total" in var:
            unc_totalbackground = Uncertainty("", is_symmetric=True)
            unc_totalbackground.values = readHist["dy"]
            round_(unc_totalbackground.values)
            varHist.add_uncertainty(unc_totalbackground)
        if var == prefix+"_data":
            unc_data = Uncertainty("", is_symmetric=True)
            unc_data.values = readHist["dy"]
            round_(unc_data.values)
            varHist.add_uncertainty(unc_data)
    
        table.add_variable(varHist)
        
    if "SR_3lAnew_" in prefix:
        nbins = 51
        bin_centers_list = list(range(1, nbins + 1))
        xBins = Variable(xBin_name, is_independent=True, is_binned=True, units= unit)
        varHist = Variable("Variable Name", is_independent=False, is_binned=True, units="",
                       bin_centers=bin_centers_list, values=bin_centers_list)

    else:
        X = reader.read_hist_1d(prefix+varss[0])
        xBins = Variable(xBin_name, is_independent=True, is_binned=True, units= unit)
        xBins.values = X["x_edges"]
        table.add_variable(xBins)
    
    submission.add_table(table)

In [81]:
# Fig 9

getTH1Ds_noAdding(
    inputFile  = "figures_9-10.root",
    fig = "Figure 9",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_009-a.pdf",
    location = "Data on page 14",
    prefix   = "SR_3lAnew_",
    varss = ["data","prompt_WZ","prompt_VH","data_fakes","convs","prompt_ZZ","prompt_TTX","total","TChiWZ_200_160_norm","TChiWZ_600_100_norm","TChiWH_150_1_norm","TChiWH_150_25_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "Search bin number",
    unit = ""
)

TypeError: __init__() got an unexpected keyword argument 'bin_centers'

In [70]:
# Fig 10

getTH1Ds_noAdding(
    inputFile  = "plots_wz-2.root",
    fig = "Figure 10",
    pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_010-a.pdf",
    location = "Data on page 14",
    prefix   = "SR_3lB_",
    varss = ["data","prompt_WZ","prompt_VH","data_fakes","prompt_ZZ","prompt_TTX","convs", "total","TChiWH_150_1_norm","TChiWH_150_25_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 3l soft category",
    xBin_name = "min(dR(l,l))",
    unit = ""
)

In [71]:
def get2DScanFromText(txtfile, depUncs = [], pdf = None, qualifiers = [], obs = None, location = "", title = "", xlabel = "", ylabel = "", zlabel="", unitsx="GeV", unitsy="GeV", unitsz="", ttitle=""):
    print(txtfile)
    inPDF = pdf
    fig = Table(ttitle)
    fig.description = title
    fig.location    = location
    fig.keywords["observables"] = obs
    theDict = {"Mass1": 0, "Mass2": 1, "Expected, median":4, "Observed":7, "Expected, -2$\sigma$": 2, "Expected, -1$\sigma$":3, "Expected, +1$\sigma$": 5, "Expected, +2$\sigma$":6}  
    theVals = {i:[] for i in theDict.keys()}
    xvals = []
    yvals = []
    theFile = open(txtfile,"r")
    for line in theFile.readlines():
        newvals = [float(a.replace(" ", "")) for a in line.split(":")]
        if len(newvals) < 7 or any([nn > 999 for nn in newvals]): continue
        for key in theVals.keys():
            theVals[key].append(newvals[theDict[key]])
        xvals.append(newvals[theDict["Mass1"]])
        yvals.append(newvals[theDict["Mass2"]])
        round_(xvals)
        round_(yvals)
    
    ## X axis
    xvar = Variable(xlabel, is_independent=True, is_binned=False, units=unitsx)
    xvar.values = xvals
    fig.add_variable(xvar)
    yvar = Variable(ylabel, is_independent=True, is_binned=False, units=unitsy)
    yvar.values = yvals
    fig.add_variable(yvar)  
    zvars = []
    for key in theDict.keys():
        if "Mass" in key: continue 
        zvars.append(Variable(key + zlabel, is_independent=False, is_binned=False, units=unitsz))
        zvars[-1].values = theVals[key]

        for q in qualifiers:
            zvars[-1].add_qualifier(*q)
        fig.add_variable(zvars[-1])

    #fig.add_image(inPDF)
    submission.add_table(fig)


In [72]:
#Fig 11
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLNNSOSparametriczedgeWX_WZ",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_011-a.pdf",
 location = "Data from Figure 11 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 11a")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLWHWX_WH",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_011-c.pdf",
 location = "Data from Figure 11 (bottom left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 11b")

get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLSOSparametriczedgeWXWH_WHWZ0p50",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_011-d.pdf",
 location = "Data from Figure 11 (bottom right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 11c")

summary_obs_Pure_NPFull_asymp_MLNNSOSparametriczedgeWX_WZ
summary_obs_Pure_NPFull_asymp_MLWHWX_WH
summary_obs_Mixed_NPFull_asymp_MLSOSparametriczedgeWXWH_WHWZ0p50


In [73]:
# Fig 12 WZ
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLNN_WZ",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_012-a.pdf",
 location = "Data from Figure 12 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12a")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametric_WZ",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_012-b.pdf",
 location = "Data from Figure 12 (top right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12b")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_zedge_WZ",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_012-c.pdf",
 location = "Data from Figure 12 (bottom left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12c")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX_WZ",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_012-d.pdf",
 location = "Data from Figure 12 (bottom right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12d")

summary_obs_Pure_NPFull_asymp_MLNN_WZ
summary_obs_Pure_NPFull_asymp_SOSparametric_WZ
summary_obs_Pure_NPFull_asymp_zedge_WZ
summary_obs_Pure_NPFull_asymp_WX_WZ


In [74]:
# Fig 12 WH
get2DScanFromText("summary_obs_Pure_NPFull_asymp_ML_WH",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_012-a.pdf",
 location = "Data from Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12a (WH)")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_WH_WH",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_012-b.pdf",
 location = "Data from Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12b (WH)")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX_WH",
 pdf = "Data from SUS-21-008_figures/CMS-SUS-21-008_Figure_012-c.pdf",
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 12c (WH)")

summary_obs_Pure_NPFull_asymp_ML_WH
summary_obs_Pure_NPFull_asymp_WH_WH
summary_obs_Pure_NPFull_asymp_WX_WH


In [75]:
# Fig 13
get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ1p00",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_013-a.pdf",
 location = "Data from Figure 13 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 13a")

get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p00",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_013-b.pdf",
 location = "Data from Figure 13 (top right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 13b")

get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p50",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_013-c.pdf",
 location = "Data from Figure 13 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 13c")

summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ1p00
summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p00
summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p50


In [76]:
# Fig 14
get2DScanFromText("summary_byBR",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_014-b.pdf",
 location = "Data from Figure 14 (left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_1^{0}}$",
 ylabel = "BR",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 14a (combined)")

get2DScanFromText("summary_byBR_ML",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_014-b.pdf",
 location = "Figure 14 (right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_1^{0}}$",
 ylabel = "BR",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 14b (ML)")

get2DScanFromText("summary_byBR_zedge",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_014-b.pdf",
 location = "Data from Figure 14 (right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_1^{0}}$",
 ylabel = "BR",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 14b (Zedge)")

get2DScanFromText("summary_byBR_4b",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_014-b.pdf",
 location = "Data from Figure 14 (right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_1^{0}}$",
 ylabel = "BR",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 14b (4b)")

summary_byBR
summary_byBR_ML
summary_byBR_zedge
summary_byBR_4b


In [77]:
# Fig 15
get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX4bMLWH_WWWHHHhino",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_015-a.pdf",
 location = "Data from Figure 15",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 15")

summary_obs_Pure_NPFull_asymp_WX4bMLWH_WWWHHHhino


In [78]:
# Fig 16
get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametric_slep",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_016-a.pdf",
 location = "Data from Figure 16 (left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 16a")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametriczedge_slep",
 pdf = "SUS-21-008_figures/CMS-SUS-21-008_Figure_016-b.pdf",
 location = "Data from Figure 16 (right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Figure 16b")

summary_obs_Pure_NPFull_asymp_SOSparametric_slep
summary_obs_Pure_NPFull_asymp_SOSparametriczedge_slep


In [79]:
submission.create_files("Test",remove_old=True)